# EVALUATIONS ON THE ENSEMBLE ATTACK
Models used: ResNet, SENet

In [32]:
import pandas as pd

from src.utils import *
import sys
import csv
import re
from sklearn.metrics import accuracy_score, balanced_accuracy_score, roc_curve, auc

In [33]:
probs_res_ens1010_3_csv = '../eval/prob_ResNet_Ensemble_10_10_3dot0.csv'
probs_res_ens1010_3 = pd.read_csv(probs_res_ens1010_3_csv, header=0, engine='python')

probs_sen_ens1010_3_csv = '../eval/prob_SENet_Ensemble_10_10_3dot0.csv'
probs_sen_ens1010_3 = pd.read_csv(probs_sen_ens1010_3_csv, header=0, engine='python')

In [34]:
len(probs_res_ens1010_3)

31488

In [35]:
len(probs_sen_ens1010_3)

31488

In [36]:
# GT labels
eval_csv = '../data/df_eval_19_3s.csv'
df_eval = pd.read_csv(eval_csv)
GT_labels = df_eval.iloc[:,-1].tolist()
print(len(GT_labels))

31489


***
## Extract predicted labels in the same order as in df_eval_19_3s

In [37]:
def extract_id(file_path):
    match = re.search(r'LA_E_(\d+)', file_path)
    if match:
        return match.group(1)
    return None
    
    
def pred_probabilities(file2_path):
    # read df_eval_19
    file1_path = '../data/df_eval_19_3s.csv'
    
    file1_ids = []
    missing_idx = []
    with open(file1_path, 'r') as file1:
        csv_reader = csv.reader(file1)
        for index, row in enumerate(csv_reader):
            file_id = extract_id(row[1])
            if file_id:
                file1_ids.append((index, file_id))
    
    # read second file and store data in a dictionary
    file2_data = {}
    with open(file2_path, 'r') as file2:
        csv_reader = csv.reader(file2)
        for row in csv_reader:
            file_id = extract_id(row[0])
            if file_id:
                file2_data[file_id] = (float(row[1]), float(row[2]))
                
    output_array = []
    for index, file_id in file1_ids:
        if file_id in file2_data:
            col2, col3 = file2_data[file_id]
            output_array.append(0 if col2>col3 else 1)
        else:
            missing_idx.append(index)
                
    return output_array, missing_idx

***
## Unbalanced accuracy

In [51]:
pred_labels_res_ens1010_3, missing_res = pred_probabilities(file2_path=probs_res_ens1010_3_csv)

In [52]:
pred_labels_sen_ens1010_3, missing_sen = pred_probabilities(file2_path=probs_sen_ens1010_3_csv)

In [55]:
UA_res_ens1010_3 = accuracy_score(y_true=GT_labels[:-1], y_pred=pred_labels_res_ens1010_3)

UA_sen_ens1010_3 = accuracy_score(y_true=GT_labels[:-1], y_pred=pred_labels_sen_ens1010_3)

print(
    f'The unbalanced accuracy for ResNet on the ensemble attack 10 10 epsilon=3.0 is {UA_res_ens1010_3*100:.2f}%\n'
    f'The unbalanced accuracy for SENet on the ensemble attack 10 10 epsilon=3.0 is {UA_sen_ens1010_3*100:.2f}%')

The unbalanced accuracy for ResNet on the ensemble attack 10 10 epsilon=3.0 is 32.47%
The unbalanced accuracy for SENet on the ensemble attack 10 10 epsilon=3.0 is 54.64%


In [54]:
BA_res_ens1010_3 = balanced_accuracy_score(y_true=GT_labels[:-1], y_pred=pred_labels_res_ens1010_3)

BA_sen_ens1010_3 = balanced_accuracy_score(y_true=GT_labels[:-1], y_pred=pred_labels_sen_ens1010_3)

print(
    f'The balanced accuracy for ResNet on the ensemble attack 10 10 epsilon=3.0 is {BA_res_ens1010_3*100:.2f}%\n'
    f'The balanced accuracy for SENet on the ensemble attack 10 10 epsilon=3.0 is {BA_sen_ens1010_3*100:.2f}%')

The balanced accuracy for ResNet on the ensemble attack 10 10 epsilon=3.0 is 22.28%
The balanced accuracy for SENet on the ensemble attack 10 10 epsilon=3.0 is 64.09%
